In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import MSO_AUTO_SIZE
from pptx.dml.color import RGBColor
import logging
import math

logger = logging.getLogger(__name__)

class AdvancedSlideManager:
    def __init__(self, template_path):
        self.prs = Presentation(template_path)
        self._validate_template()
        
        # Template metrics from user verification
        self.ROWS_PER_TEXTBOX = 16
        self.CHARS_PER_ROW = 31
        self.LINE_HEIGHT = Inches(5/16)  # 5" height / 16 rows
        
        # Layout tracking
        self.current_slide = None
        self.current_textbox = None
        self.slide_count = 0
        self.textbox_count = 0
        
        # Initialize first slide
        self._add_new_slide()

    def _validate_template(self):
        """Ensure template has required slides"""
        if len(self.prs.slides) < 1:
            raise ValueError("Template must contain at least one slide")

    def _add_new_slide(self):
        """Add new dual-column slide"""
        self.current_slide = self.prs.slides.add_slide(self.prs.slide_layouts[6])
        
        # Left column (4" width, 5" height)
        left = self.current_slide.shapes.add_textbox(Inches(0.5), Inches(1), Inches(4), Inches(5))
        left.name = "textMainBullets_L"
        left.text_frame.word_wrap = True
        left.text_frame.auto_size = MSO_AUTO_SIZE.NONE
        
        # Right column (4" width, 5" height)
        right = self.current_slide.shapes.add_textbox(Inches(4.7), Inches(1), Inches(4), Inches(5))
        right.name = "textMainBullets_R"
        right.text_frame.word_wrap = True
        right.text_frame.auto_size = MSO_AUTO_SIZE.NONE
        
        self.slide_count += 1
        self.current_textbox = left
        self.textbox_count = 0  # Reset column counter for new slide

    def _split_md_to_chunks(self, md_content):
        """Split markdown into textbox-sized chunks (16 rows x 31 chars)"""
        chunks = []
        current_chunk = []
        current_line = ""
        
        for line in md_content.split('\n'):
            line = line.strip()
            if not line:
                continue
                
            # Split line into 31-character segments
            words = line.split()
            for word in words:
                if len(current_line) + len(word) + 1 > self.CHARS_PER_ROW:
                    current_chunk.append(current_line.strip())
                    current_line = ""
                    if len(current_chunk) >= self.ROWS_PER_TEXTBOX:
                        chunks.append(current_chunk)
                        current_chunk = []
                current_line += f"{word} "
            
            if current_line:
                current_chunk.append(current_line.strip())
                current_line = ""
            
            if len(current_chunk) >= self.ROWS_PER_TEXTBOX:
                chunks.append(current_chunk)
                current_chunk = []
        
        if current_chunk:
            chunks.append(current_chunk)
        
        return chunks

    def _advance_container(self):
        """Move to next column or create new slide"""
        self.textbox_count += 1
        if self.textbox_count % 2 == 0:
            self._add_new_slide()
        else:
            self.current_textbox = self.current_slide.shapes[-1]  # Right column

    def _fill_textbox(self, chunk):
        """Fill current textbox with formatted content"""
        tf = self.current_textbox.text_frame
        tf.clear()
        
        for line in chunk:
            p = tf.add_paragraph()
            p.text = line
            p.font.size = Pt(14)
            
            # Header formatting
            if line.startswith('##'):
                p.font.bold = True
                p.font.size = Pt(18)
            elif line.startswith('###'):
                p.font.italic = True
                p.font.size = Pt(16)

    def process_md(self, md_content):
        """Process markdown with precise layout control"""
        chunks = self._split_md_to_chunks(md_content)
        logger.info(f"Content requires {len(chunks)} textboxes across {math.ceil(len(chunks)/2)} slides")
        
        for chunk in chunks:
            self._fill_textbox(chunk)
            self._advance_container()

    def save(self, output_path):
        """Save final presentation"""
        self.prs.save(output_path)
        logger.info(f"Saved presentation with {self.slide_count} slides")


# Usage example
if __name__ == "__main__":
    import sys
    if len(sys.argv) != 3:
        print("Usage: python report_generator.py input.md output.pptx")
        sys.exit(1)
    
    with open(sys.argv[1], 'r') as f:
        md_content = f.read()
    
    manager = AdvancedSlideManager("fixed_template.pptx")
    manager.process_md(md_content)
    manager.save(sys.argv[2])


In [ ]:
def create_optimized_template():
    prs = Presentation()
    
    # Single-column slide (Title content)
    slide1 = prs.slides.add_slide(prs.slide_layouts[6])
    title_box = slide1.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1.5))
    title_box.name = "title_box"
    title_box.text_frame.word_wrap = False
    title_box.text_frame.auto_size = MSO_AUTO_SIZE.NONE

    # Dual-column slides (Main content)
    for _ in range(2):  # Create 2 master slides
        slide = prs.slides.add_slide(prs.slide_layouts[6])
        for idx, col in enumerate(['L', 'R']):
            left = Inches(0.5) if col == 'L' else Inches(4.7)
            textbox = slide.shapes.add_textbox(left, Inches(1.2), Inches(4), Inches(5))
            textbox.name = f"content_{col}"
            tf = textbox.text_frame
            tf.auto_size = MSO_AUTO_SIZE.NONE
            tf.word_wrap = True
            tf.vertical_anchor = MSO_VERTICAL_ANCHOR.TOP
            tf.margin_left = Inches(0.1)
            tf.margin_right = Inches(0.1)
    
    prs.save("fixed_template.pptx")


In [ ]:
md_content = """
## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts
Short-term Treasury bills (maturing <3 months)
Commercial paper from AAA-rated corporations
Money market funds with daily liquidity
Petty cash reserves for office expenses
Foreign currency holdings in major currencies
Undeposited checks from customers
Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities
Certificates of deposit (CDs)
Bankers' acceptances
Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase)
Building improvements capitalization
Machinery installation costs
Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings
Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income

"""

In [ ]:
manager = AdvancedSlideManager("fixed_template.pptx")
manager.process_md(md_content)
manager.save("financial_report.pptx")

In [ ]:
# another trial

from pptx import Presentation
from dataclasses import dataclass
from typing import List

@dataclass
class FinancialItem:
    accounting_type: str
    account_title: str
    descriptions: List[str]
    continued: bool = False

def generate_pptx(template_path: str, items: List[FinancialItem]):
    prs = Presentation(template_path)
    
    # Get all slides from template
    slides = prs.slides
    
    # Page 1 - Use first slide
    if len(slides) >= 1:
        slide_1 = slides[0]
        populate_slide_sections(slide_1, items[:3], is_first_page=True)
    
    # Page 2+ - Use subsequent slides
    item_index = 3
    for slide_num in range(1, len(slides)):
        if item_index >= len(items):
            break
            
        current_slide = slides[slide_num]
        items_to_place = items[item_index:item_index+4]  # 2 items per slide (B and C)
        populate_slide_sections(current_slide, items_to_place)
        item_index += len(items_to_place)
    
    # Handle overflow content
    if item_index < len(items):
        raise ValueError("Insufficient slides in template for content length")
    
    return prs

def populate_slide_sections(slide, items: List[FinancialItem], is_first_page=False):
    # Find placeholders using exact names from template
    placeholders = {
        'L': next((s for s in slide.shapes if s.name == "textMainBullets_L"), None),
        'R': next((s for s in slide.shapes if s.name == "textMainBullets_R"), None),
        'C': next((s for s in slide.shapes if s.name == "textMainBullets"), None)
    }
    
    if is_first_page:
        # Page 1 layout: Section C only
        if placeholders['C']:
            tf = placeholders['C'].text_frame
            tf.clear()
            for item in items:
                add_item_to_textframe(tf, item)
    else:
        # Subsequent pages: Alternate between L and R
        current_side = 'L'
        for item in items:
            ph = placeholders.get(current_side)
            if ph:
                tf = ph.text_frame
                tf.clear()
                add_item_to_textframe(tf, item)
            current_side = 'R' if current_side == 'L' else 'L'

def add_item_to_textframe(text_frame, item: FinancialItem):
    if item.accounting_type:
        p = text_frame.add_paragraph()
        p.text = item.accounting_type + (" (continued)" if item.continued else "")
        p.level = 0
    
    if item.account_title:
        p = text_frame.add_paragraph()
        p.text = item.account_title + (" (continued)" if item.continued else "")
        p.level = 1
    
    for desc in item.descriptions:
        p = text_frame.add_paragraph()
        p.text = desc
        p.level = 2

            
# Usage example
md_content = """
## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts
Short-term Treasury bills (maturing <3 months)
Commercial paper from AAA-rated corporations
Money market funds with daily liquidity
Petty cash reserves for office expenses
Foreign currency holdings in major currencies
Undeposited checks from customers
Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities
Certificates of deposit (CDs)
Bankers' acceptances
Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase)
Building improvements capitalization
Machinery installation costs
Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings
Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income
"""

# Ensure your template.pptx contains these named placeholders:
# - First slide: "textMainBullets"
# - Subsequent slides: "textMainBullets_L" and "textMainBullets_R"

parsed_items = parse_markdown(md_content)
presentation = generate_pptx("template.pptx", parsed_items)
presentation.save("financial_report.pptx")



In [ ]:
from pptx import Presentation
from pptx.util import Inches
from dataclasses import dataclass
from typing import List
import re

@dataclass
class FinancialItem:
    accounting_type: str
    account_title: str
    descriptions: List[str]
    continued: bool = False

def parse_markdown(md_content: str) -> List[FinancialItem]:
    items = []
    current_type = ""
    current_title = ""
    current_descriptions = []
    
    for line in md_content.strip().split('\n'):
        line = line.strip()
        if not line:
            continue
            
        if line.startswith('## '):
            if current_type or current_title or current_descriptions:
                items.append(FinancialItem(current_type, current_title, current_descriptions))
            current_type = line[3:].strip()
            current_title = ""
            current_descriptions = []
        elif line.startswith('### '):
            if current_title or current_descriptions:
                items.append(FinancialItem(current_type, current_title, current_descriptions))
            current_title = line[4:].strip()
            current_descriptions = []
        else:
            current_descriptions.append(line)
    
    if current_type or current_title or current_descriptions:
        items.append(FinancialItem(current_type, current_title, current_descriptions))
    
    return items

def chunk_items(items: List[FinancialItem], items_per_slide=5) -> List[List[FinancialItem]]:
    chunks = []
    current_chunk = []
    
    for item in items:
        if len(current_chunk) >= items_per_slide:
            chunks.append(current_chunk)
            current_chunk = []
        current_chunk.append(item)
    
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

def add_slide_content(slide, items):
    content_placeholder = None
    for shape in slide.shapes:
        if shape.is_placeholder and shape.placeholder_format.idx == 1:
            content_placeholder = shape
            break
    
    if not content_placeholder:
        return
    
    text_frame = content_placeholder.text_frame
    text_frame.clear()
    
    for item in items:
        if item.accounting_type:
            p = text_frame.add_paragraph()
            p.text = item.accounting_type
            p.level = 0
            if item.continued:
                p.text += " (continued)"
        
        if item.account_title:
            p = text_frame.add_paragraph()
            p.text = item.account_title
            p.level = 1
            if item.continued:
                p.text += " (continued)"
        
        for desc in item.descriptions:
            p = text_frame.add_paragraph()
            p.text = desc
            p.level = 2

def generate_pptx(template_path: str, items: List[FinancialItem]):
    prs = Presentation(template_path)
    
    # Validate template structure
    if len(prs.slide_layouts) < 2:
        raise ValueError("Template requires at least 2 slide layouts (title + content)")

    # Page 1 - Use first layout (title slide)
    slide_1 = prs.slides.add_slide(prs.slide_layouts[0])
    title_placeholder = slide_1.shapes.title
    section_a_placeholder = next((shape for shape in slide_1.shapes if shape.name == "companyBackground"), None)
    
    # Populate company background (Section A)
    if section_a_placeholder:
        section_a_placeholder.text = "Company Name\nEst. 2020\nHQ: Hong Kong\nIndustry: Financial Services"

    # Page 1 Content (Sections B/C)
    text_placeholder = next((shape for shape in slide_1.shapes if shape.name == "textMainBullets"), None)
    if text_placeholder:
        populate_placeholder(text_placeholder, items[:3])

    # Subsequent pages using second layout (content slide)
    content_layout = prs.slide_layouts[1]
    for chunk in split_content(items[3:]):
        slide = prs.slides.add_slide(content_layout)
        
        # Get left/right placeholders from template
        left_ph = next((shape for shape in slide.shapes if shape.name == "textMainBullets_L"), None)
        right_ph = next((shape for shape in slide.shapes if shape.name == "textMainBullets_R"), None)
        
        # Distribute content according to template structure
        current_side = left_ph
        for item in chunk:
            if current_side:
                populate_placeholder(current_side, [item])
                current_side = right_ph if current_side == left_ph else left_ph

    return prs


# Usage
md_content = """
## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts
Short-term Treasury bills (maturing <3 months)
Commercial paper from AAA-rated corporations
Money market funds with daily liquidity
Petty cash reserves for office expenses
Foreign currency holdings in major currencies
Undeposited checks from customers
Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities
Certificates of deposit (CDs)
Bankers' acceptances
Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase)
Building improvements capitalization
Machinery installation costs
Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings
Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income
"""

presentation = generate_pptx("template.pptx", md_content)
presentation.save("financial_report.pptx")


In [27]:
# Proper usage
md_content = """
## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts
Short-term Treasury bills (maturing <3 months)
Commercial paper from AAA-rated corporations
Money market funds with daily liquidity
Petty cash reserves for office expenses
Foreign currency holdings in major currencies
Undeposited checks from customers
Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities
Certificates of deposit (CDs)
Bankers' acceptances
Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase)
Building improvements capitalization
Machinery installation costs
Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings
Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income
"""


In [38]:
from pptx import Presentation
from pptx.util import Pt
from dataclasses import dataclass
from typing import List, Tuple
import textwrap
import logging

logging.basicConfig(level=logging.INFO)

@dataclass
class FinancialItem:
    accounting_type: str
    account_title: str
    descriptions: List[str]
    continued: bool = False
    remaining_lines: List[str] = None  # Track partially filled descriptions

class PowerPointGenerator:
    def __init__(self, template_path: str):
        self.prs = Presentation(template_path)
        self._validate_template()
        self.current_slide_index = 0
        self.current_section = 'c'
        self.ROWS_PER_SECTION = 16
        self.CHARS_PER_ROW = 31
        self.prev_item = None
        self.partial_item = None  # Track partially processed items

    def _validate_template(self):
        if len(self.prs.slides) < 4:
            logging.warning("Template has fewer than 4 slides, content may be truncated")

    def _calculate_chunk_size(self, items: List[FinancialItem]) -> Tuple[List[FinancialItem], FinancialItem]:
        """Calculate chunk with partial item handling"""
        lines_used = 0
        capacity = []
        
        for item in items:
            # Handle partial items from previous section
            if self.partial_item:
                item = self.partial_item
                self.partial_item = None

            # Calculate lines needed including remaining lines
            item_lines = 2  # Type + title
            if item.remaining_lines:
                item_lines += len(item.remaining_lines)
            else:
                item_lines += len(item.descriptions)
            
            if lines_used + item_lines > self.ROWS_PER_SECTION:
                # Split descriptions if possible
                if lines_used < self.ROWS_PER_SECTION:
                    available_lines = self.ROWS_PER_SECTION - lines_used - 2  # Reserve for type/title
                    if available_lines > 0:
                        partial_desc = (item.remaining_lines or item.descriptions)[:available_lines]
                        remaining_desc = (item.remaining_lines or item.descriptions)[available_lines:]
                        
                        # Create partial item for next section
                        self.partial_item = FinancialItem(
                            item.accounting_type,
                            item.account_title,
                            [],
                            continued=True,
                            remaining_lines=remaining_desc
                        )
                        
                        # Add partial item to current chunk
                        capacity.append(FinancialItem(
                            item.accounting_type,
                            item.account_title,
                            partial_desc,
                            item.continued
                        ))
                        lines_used += 2 + len(partial_desc)
                break
                
            lines_used += item_lines
            capacity.append(item)
            self.partial_item = None  # Reset partial after full consumption
            
        return capacity, self.partial_item

    def parse_markdown(self, md_content: str) -> List[FinancialItem]:
        """Convert markdown to structured financial items"""
        items = []
        current_type = ""
        current_title = ""
        current_descs = []
        
        for line in md_content.strip().split('\n'):
            line = line.strip()
            if not line:
                continue
                
            if line.startswith('## '):
                if current_type or current_title or current_descs:
                    items.append(FinancialItem(current_type, current_title, current_descs))
                current_type = line[3:].strip()
                current_title = ""
                current_descs = []
            elif line.startswith('### '):
                if current_title or current_descs:
                    items.append(FinancialItem(current_type, current_title, current_descs))
                current_title = line[4:].strip()
                current_descs = []
            else:
                current_descs.append(line)
                
        if current_type or current_title or current_descs:
            items.append(FinancialItem(current_type, current_title, current_descs))
            
        return items

    def _wrap_text(self, text: str) -> List[str]:
        """Wrap text to fit within character limit"""
        return textwrap.wrap(text, width=self.CHARS_PER_ROW, break_long_words=True)

    def _calculate_item_lines(self, item: FinancialItem) -> int:
        """Calculate total lines required for an item"""
        wrapped_type = self._wrap_text(item.accounting_type)
        wrapped_title = self._wrap_text(item.account_title)
        wrapped_descs = [self._wrap_text(desc) for desc in item.descriptions]
        return len(wrapped_type) + len(wrapped_title) + sum(len(desc) for desc in wrapped_descs)

    def _get_section_shape(self, slide, section: str):
        """Retrieve shape based on section and slide position"""
        if self.current_slide_index == 0:
            return next((s for s in slide.shapes if s.name == "textMainBullets"), None)
        
        section_map = {'b': 'L', 'c': 'R'}
        target_name = f"textMainBullets_{section_map[section]}"
        return next((s for s in slide.shapes if s.name == target_name), None)

    def _populate_section(self, shape, items: List[FinancialItem]):
        """Fill shape with wrapped text and track remaining lines"""
        if not shape:
            return

        tf = shape.text_frame
        tf.clear()
        tf.word_wrap = True
        current_lines = 0
        
        for item in items:
            # Handle continuation markers
            type_text = f"{item.accounting_type} (continued)" if item.continued else item.accounting_type
            title_text = f"{item.account_title} (continued)" if item.continued else item.account_title

            # Add accounting type
            type_lines = self._wrap_text(type_text)
            for line in type_lines:
                if current_lines >= self.ROWS_PER_SECTION:
                    break
                p = tf.add_paragraph()
                p.text = line
                p.font.bold = True
                p.font.size = Pt(14)
                current_lines += 1

            # Add account title
            title_lines = self._wrap_text(title_text)
            for line in title_lines:
                if current_lines >= self.ROWS_PER_SECTION:
                    break
                p = tf.add_paragraph()
                p.text = line
                p.font.italic = True
                p.font.size = Pt(12)
                current_lines += 1

            # Add descriptions (use remaining_lines if present)
            desc_lines = item.remaining_lines if item.remaining_lines else self._wrap_text('\n'.join(item.descriptions))
            for line in desc_lines:
                if current_lines >= self.ROWS_PER_SECTION:
                    # Store remaining lines for next section
                    remaining = desc_lines[desc_lines.index(line):]
                    self.partial_item = FinancialItem(
                        item.accounting_type,
                        item.account_title,
                        [],
                        continued=True,
                        remaining_lines=remaining
                    )
                    break
                p = tf.add_paragraph()
                p.text = line
                p.font.size = Pt(11)
                p.level = 1
                current_lines += 1
            else:
                self.partial_item = None  # Clear partial if all lines consumed

    def generate(self, md_content: str, output_path: str):
        """Enhanced generation with partial item tracking"""
        items = self.parse_markdown(md_content)
        item_ptr = 0
        
        while item_ptr < len(items) or self.partial_item:
            if self.current_slide_index >= len(self.prs.slides):
                logging.warning("Insufficient slides, content truncated")
                break

            slide = self.prs.slides[self.current_slide_index]
            section_shape = self._get_section_shape(slide, self.current_section)
            
            if not section_shape:
                self._advance_section()
                continue

            # Combine remaining items with partial item
            processing_items = []
            if self.partial_item:
                processing_items.append(self.partial_item)
                self.partial_item = None
            processing_items.extend(items[item_ptr:])
            
            chunk, partial = self._calculate_chunk_size(processing_items)
            self.partial_item = partial
            
            if chunk:
                self._populate_section(section_shape, chunk)
                item_ptr += len([i for i in chunk if i not in processing_items])  # Track consumed items
                self._add_continuation_marks(chunk)
            
            self._advance_section()

        self.prs.save(output_path)
        logging.info("Generation completed with improved splitting")
    
    def _add_continuation_marks(self, items: List[FinancialItem]):
        """Mark items that continue across sections"""
        if not items:
            return
        
        # Track previous item across calls
        if self.prev_item:
            if items[0].accounting_type == self.prev_item.accounting_type:
                items[0].continued = True
                if items[0].account_title == self.prev_item.account_title:
                    items[0].continued = True
        
        # Update previous item tracking
        self.prev_item = items[-1] if items else None

    def _advance_section(self):
        """Move to next section following specified order"""
        if self.current_slide_index == 0:
            self.current_slide_index += 1
            self.current_section = 'b'
        else:
            if self.current_section == 'b':
                self.current_section = 'c'
            else:
                self.current_slide_index += 1
                self.current_section = 'b'

# Usage
if __name__ == "__main__":
    generator = PowerPointGenerator("template.pptx")
    
    try:
        generator.generate(md_content, "financial_report.pptx")
    except Exception as e:
        logging.error(f"Generation failed: {str(e)}")


INFO:root:Generation completed with improved splitting
